<a href="https://colab.research.google.com/github/KamonratNg/TEST/blob/main/DAT_DAML_DAY04_04ImageClassifier_RPS_TensorFlowGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table ><tr><td valign='center' bgcolor='white'>
  <a href="https://web.facebook.com/DAT.KUSRC/" target="_blank"><img src="https://drive.google.com/uc?id=1dNBiKikzW1-osi6lleLOgSOKQ65IIfMC" height="50px"></a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</td><td valign='center' bgcolor='white'>
  <a href="https://www.ku.ac.th/" target="_blank"><img src="https://drive.google.com/uc?id=1ZfGOBmxAwg8SAhyseFziyinzxBGme78a" height="80px"></a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</td><td valign='center' bgcolor='white'>
<a href="https://www.tensorflow.org/" target="_blank"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1200px-TensorFlowLogo.svg.png" height="80px"></a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</td><td valign='center' bgcolor='white'>
  <a href="https://mike.cpe.ku.ac.th/" target="_blank"><img src="https://drive.google.com/uc?id=1s6r3iG_Slpu_NSWqdt5zBp8Z9hV0-zh6" height="50px"></a>
</td></tr></table>

---

<center><h1><b>Image Classifier for Rock, Paper, and Scissors</b></h1></center>

---
*   Acknowledgement: Most parts of this tutorail were extracted from [Machine Learning, Zero to Hero](https://www.youtube.com/watch?v=u2TjZzNuly8&t=86s), Google TensorFlow 2019.

In [1]:
print('Image Classifier for Rock, Paper, and Scissors...')
print('  Brought to you by K.Toto@MikeLab.Net')

Image Classifier for Rock, Paper, and Scissors...
  Brought to you by K.Toto@MikeLab.Net


## 1. Recall the Problem

![](https://drive.google.com/uc?id=1rJks1MAIu5a96fiJ-Ud8Vey5plS_8LCp)

Remember this? From the first tutorial where we showed a scenario of rock, paper, and scissors, and discussed how difficult it might be to create an application that recognizes hands of different shapes, sizes, ethnicities, decorations, and more.

![](https://drive.google.com/uc?id=1e--Xp5ivQkS0ff-3BnqLPJ77LA65sCR6)

We discussed how difficult it would be to write code to detect and classify these, even for something as simple as a rock, paper or scissors.

<center><img src=https://drive.google.com/uc?id=1btOZmXxqv4OcIHc5fVHoImRBalyZC93k></center>

But since then, we have looked into machine learning, and we have seen how to build neural networks, first to detect <font color=ff00ff>**patterns**</font> in raw pixels to classify them.

<center><img src=https://drive.google.com/uc?id=1vtuQNe4b5maIbbMT8rYnK0SU2tVaawxc></center>

And then to detect <font color=ff00ff>**features**</font> using <font color=ffff00>convolutions</font> (i.e., to have a <font color=ffff00>*convolution neural network*</font> <font color=00ffff>`trained`</font> <font color=00ff00>*to spot the particular features*</font> that make up an item like the [soles](https://www.google.com/search?q=sole+of+a+shoe) or the [silhouette](https://www.google.com/search?q=silhouette+of+a+shoe) of a shoe.

Here we will see how to create a neural network that is trained on data of rock, paper, and scissors, to detect and spot them.

## 2. The data

There is a dataset here that has several hundred images of rock, paper, and scissors poses. We will train a neural network with this data.

First of all, we have to download the zip files containing the data. The code to do that is as follows. One file has the training set, the other has a testing and validation set.

In [ ]:
# map my GG drive
#from google.colab import drive
#drive.mount('/content/drive/')

## --> before open the # sign, look at the destination directory at the end 
##     of these wget commands!!
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip -O '/tmp/rps.zip'
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip -O '/tmp/rps-test-set.zip'

#print('Google Drive connected..')

### Exploring the data
In Python, we can unzip a file with the zip file library, and we unzip them to a temp directory like this.

In [ ]:
import zipfile

## set your destination working (data path) directory here
#WORK_DIR = '/content/drive/My Drive/00_DAT/'
WORK_DIR = '/tmp'

def extractFile(WORK_DIR): 
  print('Extracring all zip files to ' + WORK_DIR + '/tmp') 
  local_zip = WORK_DIR + '/rps.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall(WORK_DIR + '/tmp')
  zip_ref.close()

  local_zip = WORK_DIR + '/rps-test-set.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall(WORK_DIR + '/tmp')
  zip_ref.close()
  print('WARNING: Do not forget to disable me extractFile() after first use (call)!!')

## The following line shoule be first run, only ONCE
#extractFile(WORK_DIR)

print('zipfile imported..')
print('\t>>Warning: Bofore proceeding to the next code block,\n\t  do not forget to configure your WORK_DIR //data// path..')

This will create folders with sub-folders of each of our categories. When training in TensorFlow using an <font color=ff00ff>image data generator</font>, it will automatically label the images based on the name of their parent directory. So, we here don't need to create labels for the images.

<center><img src=https://drive.google.com/uc?id=1cl4LKGFi3WY70RC0XJ4uNoN_l-POnhdz></center>

In [ ]:
import os

rock_dir = os.path.join(WORK_DIR + '/tmp/rps/rock')
paper_dir = os.path.join(WORK_DIR + '/tmp/rps/paper')
scissors_dir = os.path.join(WORK_DIR +'/tmp/rps/scissors')

print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))

rock_files = os.listdir(rock_dir)
print(rock_files[:5])

paper_files = os.listdir(paper_dir)
print(paper_files[:5])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:5])
print('Done..')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 10
nbPix = 1

next_rock = [os.path.join(rock_dir, fname) for fname in rock_files[pic_index-nbPix:pic_index]]
next_paper = [os.path.join(paper_dir, fname) for fname in paper_files[pic_index-nbPix:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) for fname in scissors_files[pic_index-nbPix:pic_index]]

for i, img_path in enumerate(next_rock + next_paper + next_scissors):
  print(img_path)
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

print('Done..')

###Preparing the training data
So, we will achieve that with this code. This creates an image data generator that generates images for the training from the directory that they were downloaded. We can set up something called a <font color=ff00ff>training generator</font>, which as its name suggests, creates traing data from that.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = WORK_DIR + '/tmp/rps'
training_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (150, 150),
    class_mode = 'categorical'
)
print('Done..')

---
###Advanced Topic: Image augmentation test script

You can <font color=ff00ff>script</font> this code block, first, and go back here when you do your exercise later.

In [ ]:
# DEBUG: image augmentation test using ImageDataGenerator
# Reference: https://www.kdnuggets.com/2020/02/easy-image-dataset-augmentation-tensorflow.html
from keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = WORK_DIR + '/tmp/rps'

training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      #vertical_flip=True,
      fill_mode='nearest')

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (150, 150),
    batch_size = 32,
    class_mode = 'categorical'
)
print('Done..')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plotImages(images_arr):
  fig, axes = plt.subplots(1, 3, figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip( images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
    plt.tight_layout()
    plt.show()
print('Done..')

In [ ]:
augmented_images = [train_generator[0][0][0] for i in range(3)]
plotImages(augmented_images)

###Preparing the testing data
We can do exactly the same for the test set with this code. And later, when we see the ```model.fit(...)```, we will see that we passed these in as the training and validation parameters.

In [ ]:
VALIDATION_DIR = WORK_DIR + '/tmp/rps-test-set'

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (150, 150),
    class_mode = 'categorical'
)
print('Done..')

## 3. Build the model

Now let's look at our neuron network definition. This is very like what we have seen from the last tutorial, but just with more layers. 

One reason is that the rock, paper, and scissors' images are <font color=ff00ff>more complicated</font> than the grayscale clothing we saw previously, and another is that they are <font color=ff00ff>bigger</font>.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  ## Note the input shape is the desired size of the image 150x150 with 3 bytes color
  #### This is the first convolution layer
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  #### The second convolution layer
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  #### The third convolution layer
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  #### The forth convolution layer
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  ##-------------------------------------------------------------------
  ## Dense layer as we have seen before
  ## Flatten the results to feed into a DNN
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  ## 512 neuron hidden layer
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()
#--------
from datetime import datetime
import pytz

tz_BKK = pytz.timezone('Asia/Bangkok') 
datetime_BKK = datetime.now(tz_BKK)
print("Current time:", datetime_BKK.strftime("%H:%M:%S %d/%m/%y"))
print('Done..')

We can see that our input is now <font color=00ffff>150x150</font> pixel picture, with <font color=00ffff>3-byte</font> color depth. Our images are certainly bigger that they were before. And our output is a layer of the three neurons for the 3 classes of rock, paper, and scissors.

The middle part of the code is very similar to what we saw previously, but just more of it. So, here we have four <font color=ff00ff>layers of convolutions</font>, each with <font color=ff00ff>MaxPooling</font>. 

Before feeding into a dense layer, the <font color=ff00ff>dropout</font> is a little trick to improve the efficiency of a neuron network by throwing away some of the neurons.


---


We will then <font color=ff00ff>**compile**</font> the neuron network model as before with this code.

In [ ]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'rmsprop', 
              metrics = ['accuracy'])
print('Done..')

And then we can fit the data with the <font color=00ffff>```model.fit(...)```</font> call. Note that we don't have labels, as we are using the generator. it's inferring the labels from the parent directories of both the training and the validation datasets.

## 4. Fit the model

<font color='fffff00'>**Note that the fitting process takes very long time!**</font>


In [ ]:
# This process takes a long time, so think twice before running this code block
history = model.fit_generator(train_generator, epochs=25, steps_per_epoch=20,
                              validation_data = validation_generator, 
                              verbose = 1, validation_steps=3)
print('-------')
print('model fitting, DONE..')


So after finish, we can save the whole model to disk.

In [ ]:
#type(history.history)
#history.history.keys()

#### Think twice before running this code fragment!

In [ ]:
# Saving the model and result (aka. history) for later usage
WORK_DIR = '/tmp'

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

####---------------------------------------------------------
model.save(WORK_DIR + '/tmp/rps.h5')
fout = open(WORK_DIR + '/tmp/history.txt', 'w')
#### COMMENT the two above lines and rather use these lines
#### if you prepare the training data with 
#### image augmentation technique (advanced topic)
#model.save(WORK_DIR + '/tmp/rps_img_augmented.h5')
#fout = open(WORK_DIR + '/tmp/history_img_augmentd.txt', 'w')
####---------------------------------------------------------

m = ','.join(str(i) for i in acc)
fout.write(m+'\n')
m = ','.join(str(i) for i in val_acc)
fout.write(m+'\n')
m = ','.join(str(i) for i in loss)
fout.write(m+'\n')
m = ','.join(str(i) for i in val_loss)
fout.write(m+'\n')
fout.close()

print('Done..')

## 5. Evaluate the model

In [ ]:
WORK_DIR = '/tmp'
####---------------------------------------------------------
fin = open(WORK_DIR + '/tmp/history.txt')
#### COMMENT the two above lines and rather use these lines
#### if you prepare the training data with 
#### image augmentation technique (advanced topic)
#fin = open(WORK_DIR + '/tmp/history_img_augmentd.txt')
####---------------------------------------------------------
m = fin.read().splitlines()

acc = [float(i) for i in m[0].split(',')]
val_acc = [float(i) for i in m[1].split(',')]
loss = [float(i) for i in m[2].split(',')]
val_loss = [float(i) for i in m[3].split(',')]
print('Done')

In [ ]:
import matplotlib.pyplot as plt
import json

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

After the model has been been fitted, we'll probably get accuracy about 100% on the training data quite quickly, but with the validation data getting to about 87% accuracy. 

The event is something called <font color=ffff00>**over-fitting**</font>, which happens when the model gets really good at spotting what it has seen before, but it is not so great at generalizing.

Think about it this way, for example, if all our life, the only shoes that we had ever seen were hiking boots, we probably wouldn't recognize high heels as shoes. This can be said that we would be <font color=ffff00>over-fitting</font> <font color=ff00ff>ourselves</font>. 

There are a number of methods to avoid this, and one of them is called <font color=00ffff>image augmentation</font>. (*This [part](https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/) has been left for students to further explore.*)

### Test the model

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import tensorflow as tf

# Reload the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model(WORK_DIR + '/tmp/rps.h5')
# Show the model architecture
#model.summary()

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

##Your exercise



---


#### Exercise 1

Try editing the convolutions. Change the number of convolutions from 32 to either 16 or 64. What impact does that have on accuracy and training time?

#### Exercise 2

Remove the final convolution. What impact does that have on accuracy or training time?

#### Exercise 3

Add more convolutions. What impact does that have?

#### Exercise 4

Remove all convolutions but the first. What impact does that have? Experiment with it.


---



### Advance Topic I





---


Go back to <font color=00ffff>Advanced Topic: Image augmentation test script</font> in section **2. The data**, and do a self-learning under the lecturer's supervision to re-plot the evaluation curves for comparison with the old ones. rorganize your code blocks, including with text blocks if it is necessary, below this line.

---

In [ ]:
# your exercise here

In [ ]:
print("Hello, World")
# http://bit.ly/2lXXdw5

### Advanced Topic II





---

https://developers.google.com/codelabs/tensorflow-5-compleximages#0



---



### Advanced Topic III





---

https://developers.google.com/codelabs/tensorflow-6-largecnns#0


---



## My Playground


In [ ]:
# your code here